*Note: Some of the cells in this notebook are computationally expensive. To reduce runtime, this notebook is using a subset of the data.*

# Case Study: Sentiment Analysis

### Data Preparation

In [1]:
import pandas as pd
import numpy as np

# Reading the Data
df = pd.read_csv("Amazon_Unlocked_Mobile.csv")

# Sampling the Data to speed up computation
# df = df.sample(frac=0.1, random_state=10)

df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [2]:
# Dropping the missing values
df.dropna(inplace=True)

# Removing any 'neutral' rating equal to 3
df = df[df["Rating"] != 3]

# Encoding 4s and 5s as 1 (rated positively)
# Encoding 1s and 2s as 0 (rated negatively)
df["Positively Rated"] = np.where(df["Rating"] > 2, 1, 0)
df.head(10)

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Positively Rated
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0,1
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0,1
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0,1
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0,1
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0,1
5,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,1,I already had a phone with problems... I know ...,1.0,0
6,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,2,The charging port was loose. I got that solder...,0.0,0
7,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,2,"Phone looks good but wouldn't stay charged, ha...",0.0,0
8,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I originally was using the Samsung S2 Galaxy f...,0.0,1
11,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,This is a great product it came after two days...,0.0,1


In [3]:
# Most Ratings are positive
df["Positively Rated"].mean()

0.74826860258793226

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["Reviews"], 
                                                    df["Positively Rated"], 
                                                    random_state=0)

In [5]:
print("X_train first entry:\n", X_train.iloc[0])
print("\nX_train shape:\n", X_train.shape)

X_train first entry:
 I bought a BB Black and was deliveried a White BB.Really is not a serious provider...Next time is better to cancel the order.

X_train shape:
 (231207,)


# CountVectorizer

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# Fit the CountVectorizer to the training data
vect = CountVectorizer().fit(X_train)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

# Fit the CountVectorizer to the training Data
vect = CountVectorizer().fit(X_train)

In [8]:
vect.get_feature_names()[::2000]

['00',
 '4less',
 'adr6275',
 'assignment',
 'blazingly',
 'cassettes',
 'condishion',
 'debi',
 'dollarsshipping',
 'esteem',
 'flashy',
 'gorila',
 'human',
 'irullu',
 'like',
 'microsaudered',
 'nightmarish',
 'p770',
 'poori',
 'quirky',
 'responseive',
 'send',
 'sos',
 'synch',
 'trace',
 'utiles',
 'withstanding']

In [9]:
len(vect.get_feature_names())

53216

In [10]:
# Transform the documents in the training data to a document-term matrix
X_train_vectorized = vect.transform(X_train)

X_train_vectorized

<231207x53216 sparse matrix of type '<class 'numpy.int64'>'
	with 6117776 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.linear_model import LogisticRegression

# Train the model
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [12]:
from sklearn.metrics import roc_auc_score

# Predicting the transformed test Data
predictions = model.predict(vect.transform(X_test))
print("AUC: ", roc_auc_score(y_test, predictions))

AUC:  0.92648398605


In [13]:
# Get the feature names as numpy arrays
feature_names = np.array(vect.get_feature_names())

# Sort the cofficients of the model
sorted_coef_index = model.coef_[0].argsort()

# Finding the 10 smallest and 10 largest coefficients
print("Smallest Coefs:\n{}\n".format(feature_names[sorted_coef_index[:10]]))
print("Largest Coefs:\n{}\n".format(feature_names[sorted_coef_index[-11:-1]]))

Smallest Coefs:
['worst' 'false' 'worthless' 'junk' 'garbage' 'mony' 'useless' 'messing'
 'unusable' 'horrible']

Largest Coefs:
['lovely' 'love' 'amazing' 'perfecto' 'efficient' 'loves' 'loving'
 'excellent' 'exelente' 'excelente']



# Tfidf

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Fitting the TfidfVectorizer to the training data specifying a minimum document frequency of 5
vect = TfidfVectorizer(min_df=5).fit(X_train)
len(vect.get_feature_names())

17951

In [15]:
X_train_vectorized = vect.transform(X_train)

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

predictions = model.predict(vect.transform(X_test))

print("AUC: ", roc_auc_score(y_test, predictions))

AUC:  0.926610066675


In [16]:
feature_names = np.array(vect.get_feature_names())
sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()

print("Smallest Tfidf:\n{}\n".format(feature_names[sorted_tfidf_index[:10]]))
print("Largest Tfidf:\n{}\n".format(feature_names[sorted_tfidf_index[-11:-1]]))

Smallest Tfidf:
['commenter' 'pthalo' 'warmness' 'storageso' 'aggregration' '1300'
 '625nits' 'a10' 'submarket' 'brawns']

Largest Tfidf:
['goog' 'excellant' 'problems' 'aceptable' 'basico' 'goood' 'luis' 'epic'
 'gooood' 'batteries']



In [17]:
sorted_coef_index = model.coef_[0].argsort()

print("Smallest Coefs:\n{}\n".format(feature_names[sorted_coef_index[:10]]))
print("Largest Coefs:\n{}\n".format(feature_names[sorted_coef_index[-11:-1]]))

Smallest Coefs:
['not' 'worst' 'useless' 'disappointed' 'terrible' 'return' 'waste' 'poor'
 'horrible' 'doesn']

Largest Coefs:
['far' 'loves' 'best' 'easy' 'perfectly' 'awesome' 'amazing' 'perfect'
 'excellent' 'great']



In [18]:
# Treating the reviews by our current model
print(model.predict(vect.transform(["not an issue, phone is working",
                                   "Amazing, great phone is working"])))

[0 1]


# n-grams

In [19]:
# Fitting the CountVectorizer to the training data and specifying a minimum document frequency of 5 and extracting 1-grams and 
# 2-grams 
vect = CountVectorizer(min_df=5, ngram_range=(1, 2)).fit(X_train)

X_train_vectorized = vect.transform(X_train)
len(vect.get_feature_names())

198917

In [20]:
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)
predictions = model.predict(vect.transform(X_test))
print("AUC: ", roc_auc_score(y_test, predictions))

AUC:  0.967143758101


In [21]:
feature_names = np.array(vect.get_feature_names())
sorted_coef_index = model.coef_[0].argsort()
print("Smallest Coefs:\n{}\n".format(feature_names[sorted_coef_index[:10]]))
print("Largest Coefs:\n{}\n".format(feature_names[sorted_coef_index[-11:-1]]))

Smallest Coefs:
['no good' 'worst' 'junk' 'not good' 'not happy' 'horrible' 'garbage'
 'terrible' 'looks ok' 'nope']

Largest Coefs:
['amazing' 'great' 'no issues' 'awesome' 'exelente' 'no problems' 'perfect'
 'excellent' 'excelente' 'excelent']



In [22]:
print(model.predict(vect.transform(["not an issue, phone is working",
                                   "an issue, phone is not working"])))

[1 0]
